In [1]:
import os
import sys
import pyodbc
import numpy as np
import operator
import pandas as pd
import time
import datetime
import clv_scoring_functions
##(make sure our CLV scoring functions script is accessible)
import glob
from dateutil.relativedelta import relativedelta
import sklearn as sklearn
from sklearn import metrics
from scipy.optimize import minimize, rosen, rosen_der


# Declare our functions

In [2]:
def cleanCustomers(df, customer_columnname = 'customer_identifier'):
   
    """
    removes rows with blanks, zeroes, alpha-numeric characters and weird longer numbers
    identified as customer numbers. Finally, restores the datatype to 'int64'
    
    Parameters
    ----------
    score_pool : data frame
        Data frame with customer numbers
   
    customer_columnname : string
        Customer number column name from the data frame
        (customerno/customer_identifier/customer_no/customer_id… etc)
   
    Returns
    -------
    dataframe
        A data frame of cleaned customer numbers
    """
   
    df[customer_columnname] = df[customer_columnname].astype(str)
   
    # remove rows with customer number having zeroes and blanks
    shit_long_ints = df[customer_columnname].loc[df[customer_columnname].apply(lambda x: len(str(x)) > 19)].tolist()
    df = df.loc[~df[customer_columnname].isin(shit_long_ints)]
    # remove rows with customer number having zeroes and blanks
    shit_alphanums = df[customer_columnname].loc[df[customer_columnname].str.contains('[a-zA-Z]', regex=True)].tolist()
    df = df.loc[~df[customer_columnname].isin(shit_alphanums)]
    # remove rows with customer number having zeroes and blanks
    df = df.loc[~df[customer_columnname].isin(['', '0'])]
   
    df[customer_columnname] = df[customer_columnname].astype('float64')
    df[customer_columnname] = df[customer_columnname].astype('int64')
   
    return df

In [3]:
def SQL_Query(start_date, end_date, UID, password ):

    
#Format our date inputs  
    i = str(start_date)
    try:
        dt_start = datetime.datetime.strptime(i, '%Y/%m/%d')
    except ValueError:
        print("Incorrect start format")

    i = str(end_date)    
    try:
        dt_end = datetime.datetime.strptime(i, '%Y/%m/%d')
    except ValueError:
        print("Incorrect start format")

#Formatting the dates for SQL 
    dt_start = str(dt_start) + str('.000')
    dt_end = str(dt_end) + str('.000')
    
#Establish our connection to the server 
    conn_string = 'DRIVER=/usr/local/lib/libtdsodbc.so;SERVER=wnj-datasizesql;PORT=1433;DATABASE=Data Analytics;UID={};PWD={}'.format(UID,password)
    conn = pyodbc.connect(conn_string)
    
#Run our sql query

    query = ("""SELECT  Customer_Profile.customerno, c.customer_id,
    c.RFM, c.recency, c.frequency,c.order_count, c.T,
    c.AmountPerWeek, c.total_amount,
    c.FirstOrderDate, c.LastOrderDate
    FROM
  (SELECT b.customer_id,
    round(b.recency + b.frequency/10 + b.AmountPerWeek/100, 2) AS RFM,
    b.recency, b.frequency,b.order_count, b.T,
    b.AmountPerWeek, b.total_amount,
    b.FirstOrderDate, b.LastOrderDate
    FROM
      (

        SELECT
          a.customer_id,
          a.recency,
          a.frequency,
          a.order_count,
          a.T,
          round(a.total_amount / a.frequency, 2) AS AmountPerWeek,
          a.total_amount,
          a.FirstOrderDate,
          a.LastOrderDate
        FROM
          (
            SELECT
              customer_id,
              count(DISTINCT cast(DATEPART(WK, transaction_date) AS VARCHAR) + cast(year(transaction_date) AS VARCHAR))
                AS frequency,
              datediff(WEEK, MIN(transaction_date), MAX(
                  transaction_date)) AS recency,
              datediff(WEEK, MIN(transaction_date),
                       '{}')    AS T,
              count(distinct(transaction_id)) as order_count,
              MIN(
                  transaction_date)  AS FirstOrderDate,
              MAX(
                  transaction_date)  AS LastOrderDate,
              round(sum(tender_amount_total),
                    2)               AS total_amount
            FROM transaction_header
            WHERE
              transaction_date >= '{}'
            AND  
              transaction_date < '{}' 
              AND customer_id IS NOT NULL
            GROUP BY customer_id
          ) a
      ) b
    ) c
    inner join Customer_Profile ON c.customer_id = cast(Customer_Profile.customer_id AS int)
ORDER BY RFM DESC, AmountPerWeek DESC, total_amount DESC

    """.format(dt_end, dt_start, dt_end, ))

    start_time = time.time()

    trFrame = pd.read_sql_query(query, conn)

    conn.close()

    end_time = time.time()

    run_time = (end_time-start_time)/60

    print(run_time)
    return(trFrame)

In [4]:
def Generic_QL_Query(query, UID, password ):

#Establish our connection to the server 
    conn_string = 'DRIVER=/usr/local/lib/libtdsodbc.so;SERVER=wnj-datasizesql;PORT=1433;DATABASE=Data Analytics;UID={};PWD={}'.format(UID,password)
    conn = pyodbc.connect(conn_string)
    
#Run our sql query

    
    start_time = time.time()

    trFrame = pd.read_sql_query(query, conn)

    conn.close()

    end_time = time.time()

    run_time = (end_time-start_time)/60

    print(run_time)
    return(trFrame)

In [5]:
query = """SELECT  customerno, customer_id
    FROM Customer_Profile
    WHERE postal_preference = 'Opted-In' AND postal_validation = 'Deliverable'"""



In [ ]:
mailableFrame = Generic_QL_Query(query,'Dhananjay.Kumar','password1!')

In [ ]:
mailableFrame.customer_id.nunique()

In [ ]:
mailableFrame.customerno.nunique()

In [ ]:
frame.customer_id.nunique()

In [ ]:
frame.head()

In [ ]:
for i in list(mailableFrame.columns):
    mailableFrame[i] = mailableFrame[i].astype(int)

In [6]:
frame = pd.read_csv(r'/Users/sanjaygopinath/Documents/Data/Churn_Analysis/2018_01_01_2019_02_28.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
frame

,customer_id,RFM,recency,frequency,T,order_count,total_amount,FirstOrderDate,LastOrderDate,period
0,113610147855738448,202.00,2,1,33,2,20000.00,2018-07-17,2018-07-30,2018_01_01_2019_02_28
1,804070975,116.83,14,5,32,469,51416.44,2018-07-25,2018-11-02,2018_01_01_2019_02_28
2,25200300512,94.62,2,1,13,9,9262.08,2018-12-07,2018-12-20,2018_01_01_2019_02_28
3,9900560504,92.27,56,16,61,35,56426.00,2018-01-03,2019-01-30,2018_01_01_2019_02_28
4,9907212197,80.43,42,5,54,11,19216.66,2018-02-21,2018-12-09,2018_01_01_2019_02_28
5,501491851,80.19,14,2,56,168,13237.86,2018-02-09,2018-05-18,2018_01_01_2019_02_28
6,129209134,76.46,51,1,58,11,2545.96,2018-01-22,2019-01-19,2018_01_01_2019_02_28
7,501760943,70.66,14,4,57,8,22662.51,2018-01-31,2018-05-09,2018_01_01_2019_02_28
8,200385324,69.58,56,34,61,236,35961.47,2018-01-06,2019-02-01,2018_01_01_2019_02_28
9,9908596220,68.20,55,33,61,1000,33654.89,2018-01-06,2019-01-22,2018_01_01_2019_02_28


limit_output extension: Maximum message size of 10000 exceeded with 16901 characters

In [8]:
del frame['Unnamed: 0']

In [12]:
frame = cleanCustomers(customer_columnname='customer_id', df=frame)

In [24]:
mailableFrame = pd.read_csv('/Users/sanjaygopinath/Documents/Data/Churn_DM/March/DM Customers.csv', header=None)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
mailableFrame.rename(columns={0: 'customer_no'}, inplace=True)

In [33]:
mailableFrame

,customer_no
0,21001055
1,21002751
2,21003063
3,21003152
4,21003558
5,21003607
6,21003901
7,21004454
8,21004708
9,25757355


In [21]:
mailableFrame = cleanCustomers(customer_columnname='customer_id', df=mailableFrame)

KeyError: 'customer_id'

In [34]:
mailableFrame = cleanCustomers(customer_columnname='customer_no', df=mailableFrame)

In [ ]:
mailableFrame.to_csv('/Users/sanjaygopinath/Documents/Churn_DM/scored_customer_list.csv')

In [ ]:
frame.customer_id.nunique()

In [ ]:
frame['customer_id'] = frame['customer_id'].astype('int64')

In [39]:
scoredCustomer['DM_in'] = scoredCustomer.customer_id.isin(mailableFrame.customer_no)

In [38]:
len(set(scoredCustomer.customer_id).intersection(mailableFrame.customer_no))

3926984

In [ ]:
len(set(frame.customer_id).intersection(mailableFrame.customer_id))

In [ ]:
mailableFrame.customerno.nunique()

In [ ]:
frame.customer_id.nunique()

In [ ]:
len(set(frame.customer_id).intersection(mailableFrame.customerno))

In [ ]:
len(set(mailableFrame.customerno).intersection(frame.customer_id))

In [11]:
def cleanCustomers(df, customer_columnname = 'customer_identifier'):
   
    """
    removes rows with blanks, zeroes, alpha-numeric characters and weird longer numbers
    identified as customer numbers. Finally, restores the datatype to 'int64'
    
    Parameters
    ----------
    score_pool : data frame
        Data frame with customer numbers
   
    customer_columnname : string
        Customer number column name from the data frame
        (customerno/customer_identifier/customer_no/customer_id… etc)
   
    Returns
    -------
    dataframe
        A data frame of cleaned customer numbers
    """
   
    df[customer_columnname] = df[customer_columnname].astype(str)
   
    # remove rows with customer number having zeroes and blanks
    shit_long_ints = df[customer_columnname].loc[df[customer_columnname].apply(lambda x: len(str(x)) > 19)].tolist()
    df = df.loc[~df[customer_columnname].isin(shit_long_ints)]
    # remove rows with customer number having zeroes and blanks
    shit_alphanums = df[customer_columnname].loc[df[customer_columnname].str.contains('[a-zA-Z]', regex=True)].tolist()
    df = df.loc[~df[customer_columnname].isin(shit_alphanums)]
    # remove rows with customer number having zeroes and blanks
    df = df.loc[~df[customer_columnname].isin(['', '0'])]
   
    df[customer_columnname] = df[customer_columnname].astype('float64')
    df[customer_columnname] = df[customer_columnname].astype('int64')
   
    return df

In [ ]:
mailableFrame.head()

In [13]:
def scoring_BG(customer_df, parameter_dictionary, weeks):

    start  = time.time()



    a = parameter_dictionary['a']
    alpha = parameter_dictionary['alpha']
    b= parameter_dictionary['b']
    r = parameter_dictionary['r']
    print(a, alpha, b, r)


    #customer_df['period'] = str(i)+'_'+str(j)
    customer_df['BG_cond_purch_12'] = customer_df.apply(lambda q: clv_scoring_functions.BDF_conditional_expected_number_of_purchases_up_to_time(weeks,a,alpha, b, r, q['frequency'], q['recency'], q['T']), axis=1)
    customer_df['BG_prob_alive'] = customer_df.apply(lambda q: clv_scoring_functions.BDF_conditional_probability_alive(a,alpha, b, r, q['frequency'], q['recency'], q['T']), axis=1)
    customer_df['BG_prob_alive'] = customer_df.BG_prob_alive.astype(float)
    customer_df['BG_pred_spend'] = customer_df.BG_cond_purch_12*customer_df.AmountPerWeek




    end = time.time() 

    print("It took {} minutes to score {} customers with the following parameters \n {}".format(((end-start)/60),customer_df.shape[0], parameter_dictionary ))
    return(customer_df)      

In [ ]:
#create a dataframe with our accuracy scores for each class of customers and one for the total 

def accuracy_score(comparison_frame):
    """
    Parameters 
    -----------
    comparison_frame - dataframe 
        A data frame with the predicted bins and the actual bins
    
    Returns
    --------
    data frame 
        A data frame with the accuracy score for each class along with population size and overall population 
        accuracy score. 
    """
        
    temp_df = pd.DataFrame()
    i=0
    for group in comparison_frame.custora_bins.unique():
        i+=1
        comparison_frame_sub = comparison_frame[comparison_frame.custora_bins==group]
        temp_df.at[i,'class']=group
        temp_df.at[i,'size']=comparison_frame_sub.shape[0]

        temp_df.at[i,'acc_score']= sklearn.metrics.accuracy_score(comparison_frame_sub.group, comparison_frame_sub.custora_bins)
        temp_df.at[i,'f1_score']= sklearn.metrics.f1_score(comparison_frame_sub.group, comparison_frame_sub.custora_bins, average='weighted')



    temp_df.at[i+1,'class']='All'
    temp_df.at[i+1,'size']=comparison_frame.shape[0]

    temp_df.at[i+1,'acc_score']= sklearn.metrics.accuracy_score(comparison_frame.group, comparison_frame.custora_bins)

    del comparison_frame_sub

    return(temp_df)

In [ ]:
def custora_group_scoring(VS_df, VS_score_col):
    """
    Parameters 
    -----------
    VS_df - dataframe 
        the data frame with our scored customers (should have customer number
        or customer id - whatever we are going to use to merge this with the 
        custora results )
    VS_score_col - 'string'
        the column which contains our $ predictions for customers, include 
        quotation marks around the string
        
    
    Returns
    --------
    data frame 
        A data frame with a percentile rank, numerical rank and bins for 
        each customer (the bins are determined by custora's definition of 
        different customer classes, see their website for any changes)
    
    
    """
    
    VS_df['pred_spend_rank'] = VS_df[VS_score_col].rank()
    VS_df['pred_spend_perc'] = VS_df[VS_score_col].rank(pct=True)


    #these bins correspond to custoras categories. The dictionary corresponds to the ranges in the bins (in order)
    bins = np.array([0.0, 0.50, .75, .90, .95,.99,1.0001])
    bins_dict = {1:'low', 2:'medium', 3:'bronze', 4:'silver', 5:'gold', 6: 'platinum'}

    #we bin the percentile ranking using the bins articulated and then create a column that translates those 
    #bins to the custora group name 
    VS_df['pred_spend_bins'] = np.digitize(VS_df.pred_spend_perc, bins, right=True)
    VS_df['custora_bins'] = VS_df['pred_spend_bins'].replace(bins_dict)
    
    #print a sanity check to make sure the bins are correctly structured
    print("Please check to make sure that these bins make sense()\n",format(VS_df.custora_bins.value_counts()/VS_df.shape[0]))
    
    del VS_df['pred_spend_rank']
    del VS_df['pred_spend_perc']
    del VS_df['pred_spend_bins']
    
    return(VS_df)

    

In [ ]:
global model_results_list
    
model_results_list = list()
 

def minimizationfunc(params):
    
    
       
    params = params
    
    parameters = {'a':params[0], "alpha" : params[1], 'b' :params[2], 'r':params[3]}

    
    
    test_df_scored = scoring_BG(sampled_cumulative_DF, parameters, 104)

    
    test_df_scored['pred_spend_bins'] = np.digitize(test_df_scored.BG_pred_spend, bin_separators, right=False)

    test_df_scored['custora_bins'] = test_df_scored['pred_spend_bins'].replace(bins_dict)

    
    #compare the bins for the customers on the columns 
    scored = (1 - sklearn.metrics.accuracy_score(test_df_scored.group, test_df_scored.custora_bins))
    print("This is your (1- accuracy score) {}".format(scored))
    scored = float(scored)
    
    model_results_list.append([parameters, scored])

    return(scored)
    
    


In [ ]:

#create a list of pairwise dates to pass onto a SQL query that requires a specific format - ['2016/11/01', '2017/11/01']
#
def query_date(start_date, end_date, interval_months):



    date_list = []

    start_date = datetime.datetime.strptime(start_date, '%m/%d/%Y')
    end_date = datetime.datetime.strptime(end_date, '%m/%d/%Y')


    start_date = datetime.date(start_date.year, start_date.month, start_date.day)
    end_Date = datetime.date(end_date.year, end_date.month, end_date.day)

    current = start_date
    lag_month = start_date
    while lag_month < end_Date:
        current = lag_month
        current += relativedelta(months=interval_months)
        date_list.append([(lag_month).strftime('%Y/%m/%d'), (current).strftime('%Y/%m/%d')])
        lag_month += relativedelta(months=1)


    return(date_list)

# Bring In Our Data

## Bring in VS Transaction date summarized output -recency, frequency, age and total monetary value 

In [ ]:
date_list= [['2018/01/01', '2018/12/31']]

In [ ]:
date_list

In [ ]:
#create a dataframe with all our sql queries appended together with their data

#date_list = query_date('11/1/2017', '2/1/2019', 1 )

cumulative_DF = pd.DataFrame()

for i,j in date_list:
    temp_DF = SQL_Query(i,j, 'Dhananjay.Kumar','password1!')
    cumulative_DF = cumulative_DF.append(temp_DF)
    del temp_DF

    
#Print information that validates our query - find the first and last transaction for the new dataframe 

print("we have data from {} to {}.".format(cumulative_DF.FirstOrderDate.min(), cumulative_DF.LastOrderDate.max()))

In [ ]:
help(pd.Dataframe)

In [ ]:
/Users/sanjaygopinath/Documents/Data/Churn_Analysis/2018_01_01_2019_02_28.csv

In [ ]:
#scoring the whole population 

scoredCustomer = scoring_BG(cumulative_DF, parameters, 52) 

In [ ]:
scoredCustomer.to_csv('/Users/sanjaygopinath/Documents/Data/Email_Holdout_Analysis/2018_CLV_Scored.csv')

In [ ]:
cumulative_DF['total_amount'].sum()

In [ ]:
cumulative_DF.to_csv('Q2_Sales.csv')

In [ ]:
cumulative_DF.total_amount.sum()

In [ ]:
cumulative_DF[cumulative_DF.total_amount == 700]

In [ ]:
#Applying new loyalty tiers on customer spend so that we can categorize everyone 
# See here for the logic 
# In the calendar year you are either 
# bronze 0-$200
# silver $200-$700
#gold $700+

cumulative_DF['loyalty_tiers'] = pd.cut(cumulative_DF.total_amount, [0, 200, 700, np.inf], labels=['bronze','silver','gold'])

In [ ]:
#Optionally, save the dataframe so that we can double check the results later 

cumulative_DF[['customerno', 'customer_id', 'total_amount', 'loyalty_tiers']].to_csv(r'/Users/sanjaygopinath/Documents/Data/Responsys/2017_Loyalty_tiers.csv', index=None)

In [ ]:
cumulative_DF.sample(10)

In [ ]:
#Let's remove anyone who has negative or zero value for the amount they spent 

cumulative_DF = cumulative_DF[cumulative_DF.AmountPerWeek > 0]


## Bring in Custora data

In [ ]:
#import all our custora files from a specific file and then append the file name

#this is for spending predictions


path = r'/Users/sanjaygopinath/Documents/Data/CLV/Custora_Data/Machine_Learning/Spend/Jan_18' # use your path
all_files = glob.glob(os.path.join(path, "*.csv"))  

data = []

for csv in all_files:
    frame = pd.read_csv(csv)
    frame['filename'] = os.path.basename(csv)
    data.append(frame)

bigframe = pd.concat(data, ignore_index=True)

if 'Treatment Name' in bigframe.columns:
    del bigframe['Treatment Name']
    
#this creates a new column with the custora group portion of the file name. It has to be of the following format to 
# work (this is the format which is automatically downloaded from the custora app/website) - SG_Bronze_users_2018_01_09.csv
bigframe['group'] = bigframe['filename'].apply(lambda x: x.partition('_')[-1].partition('_')[0].lower())

In [ ]:
bigframe.group.value_counts()

# Score our population

In [14]:
#manually input our parameters

                
#It took 3.3765572985013326 minutes to score 888193 customers with the following parameters 
#parameters = {'a': 0.06075756, 'alpha': 6.2382612304687521, 'b': 0.70402410888671718, 'r': 2.8112931823730491}
#This is your (1- accuracy score) 0.6128161334304594

#fourth iteration with 296064 customers
#parameters = {'a': 0.025867786259379202, 'alpha': 9.8203401747296937, 'b': 1.5017444877131725, 'r': 1.293332834267968}
#This is your (1- accuracy score) 0.46638902399481197

#third iteration with 100k sample size 
#parameters = {'a': 0.047582270378977554, 'alpha': 9.7527545777315225, 'b': 1.2970538830503067, 'r': 0.97965271560800504}
#This is your (1- accuracy score) 0.47375

#second iteration with 100k sample size 
#parameters = {'a': 0.060757568359,  'alpha': 6.2382612304687521, 'b': 0.70402410888671718, 'r': 2.8112931823730491}             

#first iteration with 10k sample size
#parameters = {'a':.060965087890624825, "alpha" : 7.2610668945312398, 'b' :0.75478155517578061, 'r':2.7994494934082095}

#iteration with sample size of 5507770
#parameters = {'a': 0.031435172173152087,'alpha': 6.0207392676930009,'b': 3.560652837671328,'r': 0.76827969369016613}
#This is your (1- accuracy score) 0.473750.3624563117196252
#parameters = {'a': 0.0314,'alpha': 6.021,'b': 3.56,'r': 0.768}

#old
parameters = {'a':.34, "alpha" : 5.44, 'b' :.52, 'r':2.66}

#create a list of our parameters so they can be referenced in our optimization function (it needs to be an array
# not a dictionary)

params = [(v) for k,v in parameters.items()]


In [ ]:
path =r'/Users/sanjaygopinath/Documents/Churn_DM/' # use your path
allFiles = glob.glob(os.path.join(path, "*.csv"))
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    tempDF = pd.read_csv(file_, header=0)
    #tempDF.rename(columns={'customer_id':'CUSTOMER_ID'}, inplace=True)
    #tempDF = tempDF[['CUSTOMER_ID', 'order_count', 'total_amount', 'period']]
    #tempDF = tempDF[tempDF['CUSTOMER_ID'].isin(customerList)]                 
    
    list_.append(tempDF)


frame = pd.concat(list_)
frame = frame[frame['period']!='2016-11-01']

#del frame['Unnamed: 0']

In [15]:
frame['AmountPerWeek'] = frame.total_amount/frame.frequency

In [ ]:
scoredCustomerDF = pd.read_csv(r'/Users/sanjaygopinath/Documents/Data/BlackFriday_Analysis/summary2016-11-01.csv', header=0)
del scoredCustomerDF['Unnamed: 0']
scoredCustomerDF['frequency'] = scoredCustomerDF['frequency'].map({0: 1})
scoredCustomerDF['AmountPerWeek'] = scoredCustomerDF.total_amount/scoredCustomerDF.frequency
scoredCustomerDF = scoredCustomerDF[scoredCustomerDF.total_amount>0]

In [ ]:
scoredCustomerDF

In [ ]:
frame

In [16]:
#scoring the whole population 

scoredCustomer = scoring_BG(frame, parameters, 52) 

0.34 5.44 0.52 2.66


/Users/sanjaygopinath/Documents/Work_Space/Python/CLV/clv_scoring_functions.py:159: RuntimeWarning: invalid value encountered in log
  ln_hyp_term = np.log(hyp2f1(_a, _b, _c, _z))
/Users/sanjaygopinath/Documents/Work_Space/Python/CLV/clv_scoring_functions.py:167: RuntimeWarning: invalid value encountered in log
  ln_hyp_term_alt = np.log(hyp2f1(_c - _a, _c - _b, _c, _z)) + (_c - _a - _b) * np.log(1 - _z)


It took 17.32123573621114 minutes to score 6827363 customers with the following parameters 
 {'a': 0.34, 'alpha': 5.44, 'b': 0.52, 'r': 2.66}


In [ ]:
2018_01_01_2019_02_28.csv

In [40]:
scoredCustomer.to_csv(r'/Users/sanjaygopinath/Documents/Data/Churn_DM/March/scored_customers_2018_01_01_2019_02_28.csv')

In [19]:
scoredCustomer.head()

,customer_id,RFM,recency,frequency,T,order_count,total_amount,FirstOrderDate,LastOrderDate,period,AmountPerWeek,BG_cond_purch_12,BG_prob_alive,BG_pred_spend
0,113610147855738448,202.00,2,1,33,2,20000.00,2018-07-17,2018-07-30,2018_01_01_2019_02_28,20000.000,0.013350,0.003737,267.008865
1,804070975,116.83,14,5,32,469,51416.44,2018-07-25,2018-11-02,2018_01_01_2019_02_28,10283.288,0.687936,0.080684,7074.242030
2,25200300512,94.62,2,1,13,9,9262.08,2018-12-07,2018-12-20,2018_01_01_2019_02_28,9262.080,0.329907,0.052296,3055.626735
3,9900560504,92.27,56,16,61,35,56426.00,2018-01-03,2019-01-30,2018_01_01_2019_02_28,3526.625,11.833374,0.913806,41731.873043
4,9907212197,80.43,42,5,54,11,19216.66,2018-02-21,2018-12-09,2018_01_01_2019_02_28,3843.332,4.016989,0.702656,15438.622226


In [ ]:
scoredCustomerDF['LastOrderDate'].max()

In [ ]:
scoredCustomer['custora_bins'].value_counts()

In [ ]:
low         1801839
medium       900919
bronze       540552
silver       180184
gold         144147
platinum      36037

In [ ]:
scoredCustomer['']

In [ ]:
frame = frame.merge(scoredCustomer[['customer_id', 'custora_bins']], how='left', on='customer_id')

In [ ]:
subFrame = frame[~frame['custora_bins'].isna()]

In [ ]:
subFrame['period'].value_counts()`

In [ ]:
subFrame = frame[frame['custora_bins']!=7]

In [ ]:
subFrame['custora_bins'].groupby(subFrame['period']).value_counts().to_frame().to_csv('black_friday_results_2.csv')

In [ ]:
subFrame.period.value_counts()

In [ ]:
scoredCustomer['BG_pred_spend'].groupby(scoredCustomer['custora_bins']).mean()

In [ ]:
datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')

In [ ]:
((pd.to_datetime('2017-11-01')-pd.to_datetime(scoredCustomer['LastOrderDate'])))/time

In [ ]:
scoredCustomer['LastOrderDate']

In [ ]:
scoredCustomer['period'].astype('datetime') - scoredCustomer['LastOrderDate'].astype('datetime')

In [ ]:
scoredCustomer[scoredCustomer['BG_pred_spend'].isna()]

In [ ]:
custora_group_scoring(scoredCustomer, 'BG_pred_spend')

bins = np.array(range(0,10,1))/10
scoredCustomer['pred_spend_rank'] = scoredCustomer['BG_pred_spend'].rank()
scoredCustomer['pred_spend_perc'] = scoredCustomer['BG_pred_spend'].rank(pct=True)


#use this on the fully scored data set and then use it for bins on the samples 
scoredCustomer['pred_spend_bins'] = np.digitize(scoredCustomer.pred_spend_perc, bins, right=True)
#we reset the top and bottom of the bins so that they encompass all values (think about the percentiles, it is 
# easier to do this than to hope that there aren't any rounding issues)
#bin_separators[0]= 0


In [ ]:
# Bin our data based on Custora's groupings

cumulative_DF = custora_group_scoring(cumulative_DF, 'BG_pred_spend')

In [ ]:
#set the indices as your common shared customer id ('User ID'/'customerno'/'customer_ID')

#cumulative_DF['customerno'] = cumulative_DF['customerno'].astype(str)
#bigframe['User ID'] = bigframe['User ID'].astype(str)


#cumulative_DF.set_index(cumulative_DF['customerno'], inplace=True)

#bigframe.set_index(bigframe['User ID'], inplace=True)


#join the custora dataframe and our scored df  
comparison_frame = cumulative_DF.join(bigframe, how='inner')

#compare the bins for the customers on the bins 
sklearn.metrics.accuracy_score(comparison_frame.custora_bins, comparison_frame.group)

In [ ]:
sklearn.metrics.accuracy_score(comparison_frame.custora_bins, comparison_frame.group)

In [ ]:
bigframe.group.unique()

In [ ]:
top_10_custora = bigframe[bigframe['group'].isin(['platinum', 'silver', 'gold'])]

In [ ]:
top_10_custora.shape[0]/bigframe.shape[0]

In [ ]:
bigframe.shape[0]*.15

In [ ]:
1329453-1330816.5

In [ ]:
top_10_custora_list = list(top_10_custora.index)

In [ ]:
top_10_custora.shape

In [ ]:
len(top_10_custora_list)

In [ ]:
temp_DF = cumulative_DF.head(1996224)
#VS_top_10_list = list(temp_DF.index)
sum(temp_DF.index.isin(top_10_custora_list))/len(top_10_custora_list)

In [ ]:
sum(temp_DF.index.isin(top_10_custora_list))

In [ ]:
1130903/1329453

In [ ]:
cumulative_DF.columns

In [ ]:
cumulative_DF.sort_values('BG_pred_spend', ascending=False, inplace=True)

In [ ]:
comparison_frame.head()

In [ ]:
comparison_frame = comparison_frame[comparison_frame.frequency >1]

In [ ]:
comparison_frame.head()

In [ ]:
bigframe.columns

In [ ]:
custora_top_10 = bigframe['customera']

In [ ]:
comparison_frame.custora_bins.value_counts()

In [ ]:
comparison_frame.group.value_counts()

In [ ]:
score_frame4 = accuracy_score(comparison_frame)

In [ ]:
score_frame4

In [ ]:
score_frame2

In [ ]:
score_frame

In [ ]:
comparison_frame['custora_bins'].groupby(comparison_frame['group']).value_counts().to_frame().to_csv('bins.csv')

In [ ]:
sampled_cumulative_DF

In [ ]:
#this establishes the size of our sample data frame which we use for our optimization function 
#Check the minimization function, it references our sampled_cumulative_DF

sampled_cumulative_DF = comparison_frame.sample(frac=.5)
sampled_cumulative_DF.shape

In [ ]:
bins = np.array([0.0, 0.50, .75, .90, .95,.99,1.0001])
bins_dict = {1:'low', 2:'medium', 3:'bronze', 4:'silver', 5:'gold', 6: 'platinum'}


#use this on the fully scored data set and then use it for bins on the samples 
bin_separators = [(np.percentile(cumulative_DF.BG_pred_spend, int(x*100))) for x in bins]

#we reset the top and bottom of the bins so that they encompass all values (think about the percentiles, it is 
# easier to do this than to hope that there aren't any rounding issues)
bin_separators[0]= 0
bin_separators[6] = cumulative_DF.BG_pred_spend.max()+1

#use this if you are using a sample because otherwise the demarcation of bins, in absolute $ value, won't make sense 
#VS_df['pred_spend_bins'] = np.digitize(VS_df.pred_spend_perc, bins, right=False)

#the above changes to this - 
#VS_df['pred_spend_bins'] = np.digitize(VS_df.BG_cond_purch_12, bins, right=False)



In [ ]:
#run our minimization function

res = minimize(minimizationfunc, params, method='Nelder-Mead')

In [ ]:

#Sort our model results and then take the first one. Since we are minimzing and we are sorting in ascending order
# we will have the smallest (most accurate) score first and we want to take the parameters from that)
model_results_list = sorted(model_results_list, key=operator.itemgetter(1))   # Ascending order
parameters = model_results_list[0][0]
params = [(v) for k,v in parameters.items()]

In [ ]:
params

In [ ]:
model_results_list

In [ ]:
sampled_cumulative_DF

In [ ]:
params